In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow  as tf
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import utils
import get_network

In [6]:
model, sess = get_network.get_model()

NotFoundError: /projects/leelab3/image_datasets/imagenet/labels.txt; No such file or directory